Need to write some code to grab review data for restaurants in the Bellevue area.

We'll proceed with the free tier of the Google Places API.

<h1>Limitations</h1>
<ul>
    <li>Only 5 reviews per place is returned</li>
    <li>Maximum of 60 results per query for Nearby Search and Text Search</li>
    <li>Up to 100 queries per second</li>
    <li>Up to $200/month of free credit to use</li>
</ul>
Per month, we can rougly get ~5800 Place Details calls with reviews or ~11700 Text Search calls.

We'll be using Google Places API for initial discovery (name, location, ratings) and for the review data. If the search is limited to in/around Bellevue, we shouldn't exceed the monthly limits.

<h1>Places API (new)</h1>
We'll be using the most recent Places API to grab the data for this project: <a href='https://developers.google.com/maps/documentation/places/web-service/overview'>https://developers.google.com/maps/documentation/places/web-service/overview</a>

We need to grab our API key. It's generally best not to hardcode the raw API key into a notebook, so I've set mine to an environment variable.

In [1]:
import os

API_KEY = os.environ.get('GOOGLE_API_KEY')

In [ ]:
# Verify API key is loaded
API_KEY

After verifying that we've got our API key, we can continue to use the API to get our data.

We'll start by searching for places in Bellevue, WA and listing our results.

In [4]:
# Bellevue coordinates
latitude = 47.6101
longitude = -122.2015

# Endpoint for Places API
url = "https://places.googleapis.com/v1/places:searchNearby"

payload = {
    "includedTypes": ["restaurant"],
    "maxResultCount": 20,
    "locationRestriction": {
        "circle": {
            "center": {
                "latitude": latitude,
                "longitude": longitude},
            "radius": 5000.0
            }
        }
    }

headers = {
    "Content-Type": "application/json",
    "X-Goog-Api-Key": API_KEY,
    "X-Goog-FieldMask": "places.id,places.displayName,places.formattedAddress,places.location,places.rating"
}

We can look at the header X-Goog-Fieldmask to see exactly what data we can get from this. Important here are places.id and places.location.

I plan to use places.id to later get review data for each restaurant, then places.location to draw each restaurant on a map visualization.

In [5]:
import requests

response = requests.post(url, headers=headers, json=payload)
data = response.json()

In [ ]:
# response
data

{'places': [{'id': 'ChIJrcrho4dskFQROHfX02ZeENg',
   'formattedAddress': '10455 NE 8th St, Bellevue, WA 98004, USA',
   'location': {'latitude': 47.617188299999995,
    'longitude': -122.20077789999998},
   'rating': 4.5,
   'displayName': {'text': 'Din Tai Fung 鼎泰豐', 'languageCode': 'en'}},
  {'id': 'ChIJic2FvopskFQRb58asxzU22E',
   'formattedAddress': '785 116th Ave NE, Bellevue, WA 98004, USA',
   'location': {'latitude': 47.616794, 'longitude': -122.186212},
   'rating': 4.3,
   'displayName': {'text': 'Chick-fil-A', 'languageCode': 'en'}},
  {'id': 'ChIJ6R7bJwBtkFQR-XiMLLqDGME',
   'formattedAddress': '233 106th Ave NE, Bellevue, WA 98004, USA',
   'location': {'latitude': 47.6129752, 'longitude': -122.19923860000002},
   'rating': 4.5,
   'displayName': {'text': 'Tendon Kohaku', 'languageCode': 'en'}},
  {'id': 'ChIJJ6Ov_4VskFQRu3T8pcwahmY',
   'formattedAddress': '700 Bellevue Way NE #310, Bellevue, WA 98004, USA',
   'location': {'latitude': 47.6168325, 'longitude': -122.201042

In [5]:
sorted_places = sorted(data["places"], key=lambda p: p.get("rating", 0), reverse=True)

for place in sorted_places:
    name = place.get("displayName", {}).get("text", "Unknown")
    address = place.get("formattedAddress", "No address")
    rating = place.get("rating", "N/A")
    id = place.get("id", "No ID found")
    print(f"- {name} ({rating}) - {address}, /// {id}")

NameError: name 'data' is not defined

Now that I know how to grab 20 restaurants in a X meter radius, I want to collect restaurant data in a much larger area. I'll generate a grid of circles, each with 500m radius, and get my 20 restaurants from each.

In [6]:
from geopy.distance import distance
from geopy.point import Point

center = Point(47.6101, -122.2015) # Bellevue city center

ns_offsets = [-1000, 0, 1000, 2000] # offsets in meters for moving north/south
ew_offsets = [-1000, 0, 1000, 2000, 3000, 4000, 5000, 6000]

grid_points = []

for dlat in ns_offsets:
    for dlon in ew_offsets:
        # dlat and dlon are changes in latitude and longitude
        point_north_south = distance(meters=dlat).destination(center, bearing=0) # a bearing of 0 is North
        final_point = distance(meters=dlon).destination(point_north_south, bearing=90) # a bearing of 90 is East
        grid_points.append((final_point.latitude, final_point.longitude))

In [7]:
grid_points

[(47.601105019802425, -122.21479808425552),
 (47.601105790584626, -122.2015),
 (47.601105019802425, -122.18820191574447),
 (47.6011027074559, -122.17490383227025),
 (47.60109885354532, -122.16160575035862),
 (47.60109345807109, -122.14830767079087),
 (47.60108652103385, -122.1350095943483),
 (47.60107804243435, -122.12171152181222),
 (47.61009922897639, -122.21480036403571),
 (47.6101, -122.2015),
 (47.61009922897639, -122.18819963596428),
 (47.610096915905615, -122.17489927271049),
 (47.61009306078795, -122.16159891102055),
 (47.61008766362382, -122.14829855167636),
 (47.610080724413834, -122.13499819545987),
 (47.61007224315877, -122.12169784315299),
 (47.61909342397967, -122.21480264492457),
 (47.61909419524479, -122.2015),
 (47.61909342397967, -122.18819735507542),
 (47.61909111018441, -122.1748947109334),
 (47.619087253859256, -122.16159206835647),
 (47.619081855004644, -122.14828942812719),
 (47.61907491362118, -122.1349867910281),
 (47.61906642970964, -122.12168415784174),
 (47.

Now that we have our 3x3 grid of latitudes and longitudes, we'll grab all the restaurant data from GMaps.

In [22]:
all_places = []
num_requests = 0
for (clat, clon) in grid_points:
    num_requests += 1
    payload = {
        "includedTypes": ["restaurant"],
        "maxResultCount": 20,
        "locationRestriction": {
            "circle": {
                "center": {
                    "latitude": clat,
                    "longitude": clon},
                "radius": 500.0
                }
            }
        }

    headers = {
        "Content-Type": "application/json",
        "X-Goog-Api-Key": API_KEY,
        "X-Goog-FieldMask": "places.id,places.displayName,places.formattedAddress,places.location,places.rating"
    }

    response = requests.post(url, headers=headers, json=payload)
    data = response.json()
    if data:
        for place in data["places"]:
            location = place.get('location')
            lat = location.get('latitude')
            lon = location.get('longitude')
            new_place = {'displayName': place.get('displayName', {}).get('text', 'Unknown'),
                         'rating': place.get('rating', -1),
                         'formattedAddress': place.get('formattedAddress', 'Unknown'),
                         'id': place.get('id'),
                         'latitude': lat,
                         'longitude': lon}
            all_places.append(new_place)
print(f"Made {num_requests} requests.")
    

Made 32 requests.


In [23]:
all_places

[{'displayName': 'Dilettante Mocha Café at Bellefield Office Park',
  'rating': 4.4,
  'formattedAddress': 'Bellefield Office Park, Conifer Building, 1450 114th Ave SE #120, Bellevue, WA 98004, USA',
  'id': 'ChIJUaS2CnRskFQRgkVy8FFKcKE',
  'latitude': 47.597937099999996,
  'longitude': -122.18787809999999},
 {'displayName': 'Polaris Restaurant',
  'rating': 3.7,
  'formattedAddress': '11200 SE 6th St, Bellevue, WA 98004, USA',
  'id': 'ChIJtToSvHpskFQR9plO2_RX048',
  'latitude': 47.605349499999996,
  'longitude': -122.1895634},
 {'displayName': 'Rivue Bar and Grill',
  'rating': 4.1,
  'formattedAddress': '605 114th Ave SE, Bellevue, WA 98004, USA',
  'id': 'ChIJAy-pHnBskFQRYYC9N_vXHLo',
  'latitude': 47.6039125,
  'longitude': -122.1864034},
 {'displayName': 'TRES Sandwich',
  'rating': 4.7,
  'formattedAddress': '1502 145th Pl SE, Bellevue, WA 98007, USA',
  'id': 'ChIJwTsohjxskFQRZTRFHsbBl1o',
  'latitude': 47.5969828,
  'longitude': -122.15012670000002},
 {'displayName': 'Tokyo St

In [25]:
print(f"Got {len(all_places)} places:")
for place in sorted(all_places, key=lambda p: p.get("rating", -1), reverse=True):
    name = place.get('displayName')
    rating = place.get("rating", "N/A")
    print(f" - {name} ({rating})")

Got 192 places:
 - Evergreen Point Bistro (5)
 - Lunch on the Plate (5)
 - STALLION (5)
 - GREEN APPLE DELI-(Spring Deli) (5)
 - Empanadicious (5)
 - Naturalz Ice Cream (4.9)
 - Sammy’s Food Truck (4.9)
 - Master Bing (Bellevue) 煎饼师傅 (4.9)
 - Chili Master 湘巴佬 (4.9)
 - Kin Dee Bellevue (4.9)
 - La Chingona Mexican Cuisine Bellevue (4.8)
 - Pasifika Grill & Bar (4.8)
 - Cascadia Pizza Restaurant and Brewery (4.8)
 - Foody moody halal Indian Pakistani cuisine (4.8)
 - TRES Sandwich (4.7)
 - Haidilao Hot Pot Bellevue (4.7)
 - O'Bop (4.7)
 - Mox Boarding House (4.7)
 - Ramen Nori (4.7)
 - Sugee's Box Lunch Company (4.7)
 - El Maestro del Taco (4.7)
 - Fogo de Chão Brazilian Steakhouse (4.6)
 - La Mar Cocina Peruana Bellevue (4.6)
 - JOEY Bellevue (4.6)
 - STK Steakhouse (4.6)
 - Sanyoung Surf & Turf BBQ 三羊开炭 (4.6)
 - Supreme Dumplings (4.6)
 - Tokyo Stop Teriyaki (4.5)
 - Tendon Kohaku (4.5)
 - Lincoln Square South (4.5)
 - Carmine's (4.5)
 - Cinemark Reserve Lincoln Square — Dine-In 21+ (4

In [26]:
all_places

[{'displayName': 'Dilettante Mocha Café at Bellefield Office Park',
  'rating': 4.4,
  'formattedAddress': 'Bellefield Office Park, Conifer Building, 1450 114th Ave SE #120, Bellevue, WA 98004, USA',
  'id': 'ChIJUaS2CnRskFQRgkVy8FFKcKE',
  'latitude': 47.597937099999996,
  'longitude': -122.18787809999999},
 {'displayName': 'Polaris Restaurant',
  'rating': 3.7,
  'formattedAddress': '11200 SE 6th St, Bellevue, WA 98004, USA',
  'id': 'ChIJtToSvHpskFQR9plO2_RX048',
  'latitude': 47.605349499999996,
  'longitude': -122.1895634},
 {'displayName': 'Rivue Bar and Grill',
  'rating': 4.1,
  'formattedAddress': '605 114th Ave SE, Bellevue, WA 98004, USA',
  'id': 'ChIJAy-pHnBskFQRYYC9N_vXHLo',
  'latitude': 47.6039125,
  'longitude': -122.1864034},
 {'displayName': 'TRES Sandwich',
  'rating': 4.7,
  'formattedAddress': '1502 145th Pl SE, Bellevue, WA 98007, USA',
  'id': 'ChIJwTsohjxskFQRZTRFHsbBl1o',
  'latitude': 47.5969828,
  'longitude': -122.15012670000002},
 {'displayName': 'Tokyo St

Now that we have all the place information we need, we'll write this data to a spreadsheet for future use. We'll do that using pandas.

In [28]:
import pandas as pd

df = pd.DataFrame(all_places)
df.to_csv("places.csv", index=False)

So with this list of places and their IDs, we can run each ID through the Places API to each place's reviews. We'll do so using the Place Details (New) endpoint. We can only make 1,000 requests per month to keep the project free.

In [11]:
import pandas as pd

df = pd.read_csv("places.csv")

ids = df["id"].tolist()

In [ ]:
import requests
reviews_df = pd.DataFrame(columns=["name",
                                   "relativePublishTimeDescription",
                                   "rating",
                                   "text", "originalText",
                                   "authorAttribution",
                                   "publishTime",
                                   "flagContentUri", "googleMapsUri"])

headers = {
    "Content-Type": "application/json",
    "X-Goog-Api-Key": API_KEY,
    "X-Goog-FieldMask": "id,displayName,reviews"
}

rows = []
num_requests = 0
for curr_id in ids:
    url = "https://places.googleapis.com/v1/places/" + curr_id
    response = requests.get(url, headers=headers)
    num_requests += 1
    if response.status_code == 200:
        data = response.json()
        reviews = data.get("reviews", [])
        
        for r in reviews:
            rows.append({
                "name": r.get('name', 'Unknown'),
                "relativePublishTimeDescription": r.get('relativePublishTimeDescription', 'Unknown'),
                "rating": r.get('rating', 'Unknown'),
                "text": r.get("text", "No review found"),
                "originalText": r.get("originalText", "No review found"),
                "authorAttribution": r.get("authorAttribution", "No author found"),
                "publishTime": r.get("publishTime", "N/A"),
                "flagContentUri": r.get("flagContentUri", "N/A"),
                "googleMapsUri": r.get("googleMapsUri", "N/A")
            })
print(f"Made {num_requests} requests.")


Made 192 requests.


In [47]:
reviews_df = pd.DataFrame(rows)

In [48]:
reviews_df

,name,relativePublishTimeDescription,rating,text,originalText,authorAttribution,publishTime,flagContentUri,googleMapsUri
0,places/ChIJUaS2CnRskFQRgkVy8FFKcKE/reviews/Chd...,a month ago,4,{'text': 'They have very nice grab n go option...,{'text': 'They have very nice grab n go option...,"{'displayName': 'Shana Bobbitt', 'uri': 'https...",2025-05-08T21:42:41.981034Z,https://www.google.com/local/review/rap/report...,https://www.google.com/maps/reviews/data=!4m6!...
1,places/ChIJUaS2CnRskFQRgkVy8FFKcKE/reviews/Chd...,3 weeks ago,2,{'text': 'I had seen plenty of bad reviews abo...,{'text': 'I had seen plenty of bad reviews abo...,"{'displayName': 'Kelly Lee', 'uri': 'https://w...",2025-06-10T23:12:31.737849Z,https://www.google.com/local/review/rap/report...,https://www.google.com/maps/reviews/data=!4m6!...
2,places/ChIJUaS2CnRskFQRgkVy8FFKcKE/reviews/ChZ...,10 months ago,5,"{'text': 'Large space, tasty menu options, and...","{'text': 'Large space, tasty menu options, and...","{'displayName': 'Ksusha Malysheva', 'uri': 'ht...",2024-08-23T22:37:49.628667Z,https://www.google.com/local/review/rap/report...,https://www.google.com/maps/reviews/data=!4m6!...
3,places/ChIJUaS2CnRskFQRgkVy8FFKcKE/reviews/Chd...,a month ago,3,"{'text': 'Ok, so I've been here a few times an...","{'text': 'Ok, so I've been here a few times an...","{'displayName': 'Mick Thornton', 'uri': 'https...",2025-06-02T18:41:12.863221Z,https://www.google.com/local/review/rap/report...,https://www.google.com/maps/reviews/data=!4m6!...
4,places/ChIJUaS2CnRskFQRgkVy8FFKcKE/reviews/ChZ...,2 months ago,5,{'text': 'uh oh found another favorite that's ...,{'text': 'uh oh found another favorite that's ...,"{'displayName': 'Jisoo Jeong', 'uri': 'https:/...",2025-04-16T23:23:13.411396Z,https://www.google.com/local/review/rap/report...,https://www.google.com/maps/reviews/data=!4m6!...
...,...,...,...,...,...,...,...,...,...
931,places/ChIJR4_zXR9tkFQRR7gn-iENrC8/reviews/Chd...,10 months ago,4,{'text': 'Shocked and pleasantly surprised tha...,{'text': 'Shocked and pleasantly surprised tha...,"{'displayName': 'S S', 'uri': 'https://www.goo...",2024-08-12T00:46:35.369292Z,https://www.google.com/local/review/rap/report...,https://www.google.com/maps/reviews/data=!4m6!...
932,places/ChIJR4_zXR9tkFQRR7gn-iENrC8/reviews/Chd...,7 months ago,5,{'text': 'This is definitely the BEST Mexican ...,{'text': 'This is definitely the BEST Mexican ...,"{'displayName': 'Jeffrey Chappon', 'uri': 'htt...",2024-11-12T22:07:36.423790Z,https://www.google.com/local/review/rap/report...,https://www.google.com/maps/reviews/data=!4m6!...
933,places/ChIJR4_zXR9tkFQRR7gn-iENrC8/reviews/ChZ...,a year ago,2,{'text': 'I ordered the birria torta and two a...,{'text': 'I ordered the birria torta and two a...,"{'displayName': 'Tricia Ward', 'uri': 'https:/...",2023-12-30T01:52:03.339267Z,https://www.google.com/local/review/rap/report...,https://www.google.com/maps/reviews/data=!4m6!...
934,places/ChIJR4_zXR9tkFQRR7gn-iENrC8/reviews/Chd...,6 months ago,3,"{'text': 'food here is 3/5, quesadilla is fine...","{'text': 'food here is 3/5, quesadilla is fine...","{'displayName': 'eve', 'uri': 'https://www.goo...",2024-12-26T17:38:11.917088Z,https://www.google.com/local/review/rap/report...,https://www.google.com/maps/reviews/data=!4m6!...


In [49]:
# Cleaning up the name column to display a human-readable ID and raw strings for review text
reviews_df["name"] = reviews_df["name"].str.split("/").str[1]
reviews_df["text"] = reviews_df["text"].apply(lambda x: x.get("text") if isinstance(x, dict) else x)
reviews_df

,name,relativePublishTimeDescription,rating,text,originalText,authorAttribution,publishTime,flagContentUri,googleMapsUri
0,ChIJUaS2CnRskFQRgkVy8FFKcKE,a month ago,4,They have very nice grab n go options but are ...,{'text': 'They have very nice grab n go option...,"{'displayName': 'Shana Bobbitt', 'uri': 'https...",2025-05-08T21:42:41.981034Z,https://www.google.com/local/review/rap/report...,https://www.google.com/maps/reviews/data=!4m6!...
1,ChIJUaS2CnRskFQRgkVy8FFKcKE,3 weeks ago,2,I had seen plenty of bad reviews about the cus...,{'text': 'I had seen plenty of bad reviews abo...,"{'displayName': 'Kelly Lee', 'uri': 'https://w...",2025-06-10T23:12:31.737849Z,https://www.google.com/local/review/rap/report...,https://www.google.com/maps/reviews/data=!4m6!...
2,ChIJUaS2CnRskFQRgkVy8FFKcKE,10 months ago,5,"Large space, tasty menu options, and stylish d...","{'text': 'Large space, tasty menu options, and...","{'displayName': 'Ksusha Malysheva', 'uri': 'ht...",2024-08-23T22:37:49.628667Z,https://www.google.com/local/review/rap/report...,https://www.google.com/maps/reviews/data=!4m6!...
3,ChIJUaS2CnRskFQRgkVy8FFKcKE,a month ago,3,"Ok, so I've been here a few times and it's a n...","{'text': 'Ok, so I've been here a few times an...","{'displayName': 'Mick Thornton', 'uri': 'https...",2025-06-02T18:41:12.863221Z,https://www.google.com/local/review/rap/report...,https://www.google.com/maps/reviews/data=!4m6!...
4,ChIJUaS2CnRskFQRgkVy8FFKcKE,2 months ago,5,uh oh found another favorite that's far far fr...,{'text': 'uh oh found another favorite that's ...,"{'displayName': 'Jisoo Jeong', 'uri': 'https:/...",2025-04-16T23:23:13.411396Z,https://www.google.com/local/review/rap/report...,https://www.google.com/maps/reviews/data=!4m6!...
...,...,...,...,...,...,...,...,...,...
931,ChIJR4_zXR9tkFQRR7gn-iENrC8,10 months ago,4,Shocked and pleasantly surprised that a food t...,{'text': 'Shocked and pleasantly surprised tha...,"{'displayName': 'S S', 'uri': 'https://www.goo...",2024-08-12T00:46:35.369292Z,https://www.google.com/local/review/rap/report...,https://www.google.com/maps/reviews/data=!4m6!...
932,ChIJR4_zXR9tkFQRR7gn-iENrC8,7 months ago,5,This is definitely the BEST Mexican food I’ve ...,{'text': 'This is definitely the BEST Mexican ...,"{'displayName': 'Jeffrey Chappon', 'uri': 'htt...",2024-11-12T22:07:36.423790Z,https://www.google.com/local/review/rap/report...,https://www.google.com/maps/reviews/data=!4m6!...
933,ChIJR4_zXR9tkFQRR7gn-iENrC8,a year ago,2,I ordered the birria torta and two al pastor t...,{'text': 'I ordered the birria torta and two a...,"{'displayName': 'Tricia Ward', 'uri': 'https:/...",2023-12-30T01:52:03.339267Z,https://www.google.com/local/review/rap/report...,https://www.google.com/maps/reviews/data=!4m6!...
934,ChIJR4_zXR9tkFQRR7gn-iENrC8,6 months ago,3,"food here is 3/5, quesadilla is fine, but burr...","{'text': 'food here is 3/5, quesadilla is fine...","{'displayName': 'eve', 'uri': 'https://www.goo...",2024-12-26T17:38:11.917088Z,https://www.google.com/local/review/rap/report...,https://www.google.com/maps/reviews/data=!4m6!...


In [50]:
reviews_df.to_csv("reviews.csv")